In [ ]:
!pip install transformers

In [ ]:
!pip install sentence_transformers

In [ ]:
def compare_sentences(original_sentence, sentence_list):
      original_embedding = sbert_model.encode(original_sentence)
      compare_embedding = sbert_model.encode(sentence_list)

      sim = util.cos_sim(original_embedding, compare_embedding).tolist()[0]
      # sim = util.dot_score(original_embedding, compare_embedding).tolist()[0]

      return sim


In [ ]:
data_dir = '/content/'

similar_answers = data_dir + '1.all_questions.csv'
similar_answers_score = data_dir + '2.all_questions_ similarity_scores.csv'

In [ ]:
# suppress warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import json
import pandas as pd

def calculate_similarity_score(data_source): #, input, output):

  global similarity_json
  global question_answers_dict

  # read similar_answers file
  df = pd.read_csv(similar_answers)

  df_out = pd.read_csv(similar_answers_score)
  # df_out = pd.read_csv(input)

  df['question_keyword'] = df['question_keyword'].str.lower()
  df_out['question_keyword'] = df_out['question_keyword'].str.lower()

  df_master = pd.DataFrame()



  # group df by id and question_keyword
  df_grouped = df.groupby(['id', 'question_keyword'])

  #print(df_grouped.groups)

  # iterate over the groups
  for name, group in df_grouped:


    print(name)
    id = name[0]
    question_keyword = name[1]
    # print(question_keyword)

    # get all the question_approaches
    question_approaches = group['question_approach'].unique()

    # loop over the question_approaches
    for question_approach in question_approaches:
      # get the question and answer with question_approach as Ask deeper question

      score_suffix = ''
      question_column = 'question'
      answer_column = 'answer'

      if data_source == 'qaqa':
        score_suffix = '_qaqa'
        question_column = 'new_question'
        answer_column = 'new_answer'

        # This is the base answer, so always same
        if question_approach == 'Ask deeper question':
          answer_column = 'answer'

      question =         group[group['question_approach'] == question_approach][question_column].values[0]
      answer =           group[group['question_approach'] == question_approach][answer_column].values[0]

      # print(question_keyword)
      # print(question)
      # print(answer)

      if question_approach == 'Ask deeper question':
        question_answers_dict['base-answer'] = answer

      if question_approach == 'Challenge with how':
        question_answers_dict['how-question'] = question
        question_answers_dict['how-answer'] = answer

      if question_approach == 'Challenge with why':
        question_answers_dict['why-question'] = question
        question_answers_dict['why-answer'] = answer

      if question_approach == 'Challenge with really':
        question_answers_dict['really-question'] = question
        question_answers_dict['really-answer'] = answer


    # print the question_answers_dict in readable format with indentation and newlines
    question_answers_json = json.dumps(question_answers_dict, indent=4)
    # print(question_answers_json)

    challenge_types = ['how', 'why', 'really']

    for challenge_type in challenge_types:
      # get the similarity score for the question and answer
      similarity_score_questions, similarity_score_answers = get_similarity_score(question_answers_dict, challenge_type)

      question_approach = 'Challenge with '+challenge_type

      # get the row from df_out that matches with id, question_keyword and question_approach
      df_out_row = df_out.loc[(df_out['id'] == id) & (df_out['question_keyword'] == question_keyword) & (df_out['question_approach'] == question_approach)]

      # update the similarity score of df_out_row
      df_out_row['sim_ques_ans'+score_suffix] = similarity_score_questions[0]
      df_out_row['sim_ques_how'+score_suffix] = similarity_score_questions[1]
      df_out_row['sim_ques_why'+score_suffix] = similarity_score_questions[2]
      df_out_row['sim_ques_really'+score_suffix] = similarity_score_questions[3]

      df_out_row['sim_ans_base'+score_suffix] = similarity_score_answers[0]
      df_out_row['sim_ans_how'+score_suffix] = similarity_score_answers[1]
      df_out_row['sim_ans_why'+score_suffix] = similarity_score_answers[2]
      df_out_row['sim_ans_really'+score_suffix] = similarity_score_answers[3]

      # update the df_out_row in df_out
      df_out.loc[(df_out['id'] == id) & (df_out['question_keyword'] == question_keyword) & (df_out['question_approach'] == question_approach)] = df_out_row

      df_out.to_csv(similar_answers_score, index=False)

      # somehow df_out is not being written into file.
      df_master = df_master.append(df_out_row, ignore_index=True)
      #print(df_master)

    # break
    # if id > 1:
      # break
  # df_master.to_csv(output, index=False)


In [ ]:
from sentence_transformers import util
from sentence_transformers import SentenceTransformer


sbert_model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
sbert_model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')
sbert_model = SentenceTransformer('all-mpnet-base-v2')
sbert_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')  # ../3rd_models/multi-qa-MiniLM-L6-cos-v1



In [ ]:
%%sh
# rm '/content/1.all_questions.csv'
rm '/content/2.all_questions_ similarity_scores.csv'

In [ ]:
question_answers_dict = {'base-answer': '',
                   'how-question': '',
                   'why-question': '',
                   'really-question': '',
                   'how-answer': '',
                   'why-answer': '',
                   'really-answer': ''
                   }

similarity_json = {'how-question': {'simiarity_score_with_how_answer':0.0, 'similarity_score_with_how_question': 0.0, 'similarity_score_with_why_question': 0.0, 'similarity_score_with_really_question': 0.0},
                   'why-question': {'simiarity_score_with_why_answer':0.0, 'similarity_score_with_how_question': 0.0, 'similarity_score_with_why_question': 0.0, 'similarity_score_with_really_question': 0.0},
                   'really-question': {'simiarity_score_with_really_answer':0.0, 'similarity_score_with_how_question': 0.0, 'similarity_score_with_why_question': 0.0, 'similarity_score_with_really_question': 0.0},
                    'how-answer': {'simiarity_score_with_base_answer':0.0, 'similarity_score_with_how_answer': 0.0, 'similarity_score_with_why_answer': 0.0, 'similarity_score_with_really_answer': 0.0},
                    'why-answer': {'simiarity_score_with_base_answer':0.0, 'similarity_score_with_how_answer': 0.0, 'similarity_score_with_why_answer': 0.0, 'similarity_score_with_really_answer': 0.0},
                    'really-answer': {'simiarity_score_with_base_answer':0.0, 'similarity_score_with_how_answer': 0.0, 'similarity_score_with_why_answer': 0.0, 'similarity_score_with_really_answer': 0.0}
                   }


def get_similarity_score(question_answers_dict, challenge_type):
  original = question_answers_dict[challenge_type+"-question"]
  sentences_list = [question_answers_dict[challenge_type+"-answer"], question_answers_dict["how-question"], question_answers_dict["why-question"], question_answers_dict["really-question"]]

  similarity_score_questions = compare_sentences(original, sentences_list)
  # print(similarity_score_questions)

  original = question_answers_dict[challenge_type+"-answer"]
  sentences_list = [question_answers_dict["base-answer"], question_answers_dict["how-answer"], question_answers_dict["why-answer"], question_answers_dict["really-answer"]]

  similarity_score_answers = compare_sentences(original, sentences_list)

  return similarity_score_questions, similarity_score_answers


In [ ]:
calculate_similarity_score('base')
calculate_similarity_score('qaqa')
